<a href="https://colab.research.google.com/github/PS-The-SocioPath/Sentiment-Analysis-using-Machine-Learning/blob/main/3_Final_Project_(with%20NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
### Importing the required libraries
import pandas as pd

### Data Importing
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-O9oVRtEVurfZerjJcuF49fLHOt1O4zNpifUJcoal_KuqvBxZvrrhR-XRUZ4W3REv4KzlMWRMaJQ9/pub?gid=1017866361&single=true&output=csv')

df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [43]:
### Data Inspection

print(df.describe())
print(df.info())

            overall  unixReviewTime
count  10261.000000    1.026100e+04
mean       4.488744    1.360606e+09
std        0.894642    3.779735e+07
min        1.000000    1.095466e+09
25%        4.000000    1.343434e+09
50%        5.000000    1.368490e+09
75%        5.000000    1.388966e+09
max        5.000000    1.405987e+09
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      10261 non-null  object
 1   asin            10261 non-null  object
 2   reviewerName    10234 non-null  object
 3   helpful         10261 non-null  object
 4   reviewText      10254 non-null  object
 5   overall         10261 non-null  int64 
 6   summary         10261 non-null  object
 7   unixReviewTime  10261 non-null  int64 
 8   reviewTime      10261 non-null  object
dtypes: int64(2), object(7)
memory usage: 721.6+ KB
None


In [44]:
### Initial data cleaning

print(df.reviewerName.isnull())

### The df.info step shows 27 missing values in name, on inspecting the dataset, I did find some inconsistencies in names such as initials or some letters, let's clean the
### data accordingly.

### 1. Converting to the Name/Title case
df['reviewerName'] = df['reviewerName'].str.title()
print(df['reviewerName'])

import re

### 2. Removing any unrequired symbols using regex
df['reviewerName'] = df['reviewerName'].str.replace(r'[^a-zA-Z\s]','', regex=True)
print(df['reviewerName'])

### 3. Filling missing/NA values with Unknown/Anonymous User
df['reviewerName'] = df['reviewerName'].fillna('Anonymous User')
df['reviewerName'][41]

### 4. Handling initials - can also be done using apply.lambda but it seems a bit
### advanced here since there are many different sorts of entries, lets check df now
print(df.info())

### this shows that df now only has 7 problematic entries in reviewText

0        False
1        False
2        False
3        False
4        False
         ...  
10256    False
10257    False
10258    False
10259    False
10260    False
Name: reviewerName, Length: 10261, dtype: bool
0        Cassandra Tu "Yeah, Well, That'S Just Like, U...
1                                                    Jake
2                           Rick Bennette "Rick Bennette"
3                               Rustybill "Sunday Rocker"
4                                           Sean Maslanka
                               ...                       
10256                                     Lonnie M. Adams
10257                                  Michael J. Edelman
10258                                    Michael L. Knapp
10259                             Rick Langdon "Scriptor"
10260                                     Theterrorbeyond
Name: reviewerName, Length: 10261, dtype: object
0        Cassandra Tu Yeah Well ThatS Just Like U
1                                            Jake
2

In [47]:
### Cleaning reviewText

# print(df['reviewText'].isnull())

df['reviewText'] = df['reviewText'].fillna('No Review')
print(df['reviewText'][9306])

### Let's check once
print(df.info())

### The output seems that our data has been cleaned in a basic way, some steps
### like removing duplicates and validating the format
### remain which I will execute now

### Before that it is essential to decapitalize everything as it helps in
### tokenization

No Review
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      10261 non-null  object
 1   asin            10261 non-null  object
 2   reviewerName    10261 non-null  object
 3   helpful         10261 non-null  object
 4   reviewText      10261 non-null  object
 5   overall         10261 non-null  int64 
 6   summary         10261 non-null  object
 7   unixReviewTime  10261 non-null  int64 
 8   reviewTime      10261 non-null  object
dtypes: int64(2), object(7)
memory usage: 721.6+ KB
None


In [49]:
### Data Preprocessing

### Lowercasing
df['reviewerName'] = df['reviewerName'].str.lower().str.strip()
df['reviewText'] = df['reviewText'].str.lower().str.strip()
df['summary'] = df['summary'].str.lower().str.strip()

# print(df)



TypeError: Series.drop_duplicates() got an unexpected keyword argument 'subset'